# Work in progress

This notebook is not finished. 
Neither the basic RNN or LSTM is implemented yet.


In [ ]:
from keras.datasets import imdb
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import numpy as np

corpus_size = 30000
maxlen = 300

# Since the curent version (1.16.4) of numpy sets 'allow_pickle' to False by default,
# we need to overwrite this parameter to be able to load the dataset into memory.
# We should, obviously, reset the default parameters later

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=corpus_size)

# restore np.load for future normal usage
np.load = np_load_old


# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(train_data, maxlen = maxlen, padding = 'post')
X_test = pad_sequences(test_data, maxlen = maxlen, padding = 'post')

X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

def gru_model():
    model = Sequential()
    model.add(GRU(50, input_shape = (300,1), return_sequences = True))
    model.add(GRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model
    
model = gru_model()

%%time
model.fit(X_train, train_labels, batch_size = 100, epochs = 10, verbose = 0)

scores = model.evaluate(X_test, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
